## Notes:
1. Model Used: Simple Dense Neural Network trained on TFIDF Vectors
2. Learning Rate: 0.001
3. Criterion: BCELoss
4. Optimizer: Adam

## Step 1: Imports

In [1]:
# General Imports
import copy
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Pytorch Imports
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader

print(os.listdir('../input/'))

['sample_submission.csv', 'train.csv', 'test.csv']


## Step 2: Importing the Train CSV and splitting into train and val data

In [2]:
trainDF = pd.read_csv('../input/train.csv')
trainDF.head()

,Id,review,sentiment
0,0,"I have done a lot of international travel, bot...",1
1,1,One of the most frightening game experiences e...,1
2,2,I was amazingly impressed by this movie. It co...,1
3,3,"This film is stale, and misses the mark. It is...",0
4,4,At last!! Sandra Bullock is indeed a beautiful...,1


In [3]:
train_x, val_x, train_y, val_y = train_test_split(trainDF['review'], trainDF['sentiment'], test_size=0.1, shuffle=True)

## Step 3: *Most Important*, Creating the TFIDF Vectors for the data

In [4]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=25000)
tfidf_vect.fit(trainDF['review'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xval_tfidf = tfidf_vect.transform(val_x)

In [5]:
# Converting the Sparse matrix into a numpy array
xtrain_tfidf = xtrain_tfidf.toarray()
xval_tfidf = xval_tfidf.toarray()
# Converting pandas Series into numpy array
train_y = np.array(train_y)
val_y = np.array(val_y)

## Step 4: Converting the Numpy arrays into Pytorch Datasets

In [6]:
train_dataset = TensorDataset(torch.from_numpy(xtrain_tfidf).double(), torch.from_numpy(train_y).double())
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64)

val_dataset = TensorDataset(torch.from_numpy(xval_tfidf).double(), torch.from_numpy(val_y).double())
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=64)

dataloaders = {'train': train_dataloader, 'val': val_dataloader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

In [7]:
# Hyper Parameters
epochs = 30
input_dim = 25000
output_dim = 1
lr_rate = 0.001

## Step 5: Defining a Simple Dense Model

In [8]:
model = nn.Sequential(nn.Linear(input_dim, 2048),
                      nn.Dropout(0.5),
                      nn.ReLU(),
                      nn.Linear(2048, 256),
                      nn.Dropout(0.5),
                      nn.ReLU(),
                      nn.Linear(256, output_dim),
                      nn.Sigmoid()).double()

In [9]:
# Criterion and Optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)

## Step 6: Defining the Train Function

In [10]:
def train_model(model):
    model = model.cuda()
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(int(epochs)):
        train_loss = 0
        val_loss = 0
        val_acc = 0
        model.train()
        for inputs, labels in train_dataloader:
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        else:
            model.eval()
            num_correct = 0
            for inputs, labels in val_dataloader:
                inputs, labels = inputs.cuda(), labels.cuda()
                outputs = model(inputs)
                predictions = torch.round(outputs.squeeze())
                loss = criterion(predictions, labels)
                
                val_loss += loss.item()
                equals = (predictions == labels.data)
    
                num_correct += torch.sum(equals.data).item()
            
            val_acc = num_correct / len(val_dataset)
            if val_acc > best_acc:
                best_acc = val_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print('---------Epoch {} -----------'.format(epoch))
        print('Train Loss: {:.6f} Val Loss: {:.6f} Val Accuracy: {:.6f}'.format(
                 train_loss/len(train_dataset), val_loss/len(val_dataset), val_acc))
        
    model.load_state_dict(best_model_wts)
    return model

In [11]:
model = train_model(model)

---------Epoch 0 -----------
Train Loss: 0.009009 Val Loss: 0.008801 Val Accuracy: 0.873429
---------Epoch 1 -----------
Train Loss: 0.008401 Val Loss: 0.008789 Val Accuracy: 0.891714
---------Epoch 2 -----------
Train Loss: 0.008283 Val Loss: 0.008736 Val Accuracy: 0.899429
---------Epoch 3 -----------
Train Loss: 0.008223 Val Loss: 0.008849 Val Accuracy: 0.891143
---------Epoch 4 -----------
Train Loss: 0.008188 Val Loss: 0.008824 Val Accuracy: 0.892857
---------Epoch 5 -----------
Train Loss: 0.008162 Val Loss: 0.008844 Val Accuracy: 0.891143
---------Epoch 6 -----------
Train Loss: 0.008142 Val Loss: 0.008811 Val Accuracy: 0.892286
---------Epoch 7 -----------
Train Loss: 0.008133 Val Loss: 0.008794 Val Accuracy: 0.890000
---------Epoch 8 -----------
Train Loss: 0.008136 Val Loss: 0.009025 Val Accuracy: 0.878571
---------Epoch 9 -----------
Train Loss: 0.008122 Val Loss: 0.008831 Val Accuracy: 0.893429
---------Epoch 10 -----------
Train Loss: 0.008124 Val Loss: 0.008792 Val Accura

## Step 7: Load the test csv and create the test dataset

In [12]:
test_df = pd.read_csv('../input/test.csv')
xtest_tfidf =  tfidf_vect.transform(test_df['review'])
xtest_tfidf = xtest_tfidf.toarray()

In [13]:
test_y = np.zeros(xtest_tfidf.shape[0])

In [14]:
#predictions = classifier.predict(xtest_tfidf)
test_dataset = TensorDataset(torch.from_numpy(xtest_tfidf), torch.from_numpy(test_y))
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=64)

## Step 8: Predicting on the Test Dataset

In [15]:
def predict(model, test_dataloader):
    model.eval()
    predictions = []
    for inputs, _ in test_dataloader:
        inputs = inputs.cuda()
        output = model(inputs)
        preds = torch.round(output)
        predictions.extend([p.item() for p in preds])
    return predictions

predictions = predict(model, test_dataloader)

In [16]:
sub_df = pd.DataFrame()
sub_df['Id'] = test_df['Id']
sub_df['sentiment'] = [int(p) for p in predictions]

In [17]:
sub_df.head()

,Id,sentiment
0,35000,1
1,35001,0
2,35002,0
3,35003,0
4,35004,0


In [18]:
sub_df.to_csv('my_submission.csv', index=False)
